In [1]:
# Install missing library
!pip install openpyxl


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd

# Data

In [3]:
# Now that we've installed the necessary library, let's try loading the data again
data = pd.read_excel('WL.xlsx')

# Preview the data
data.head()

,Unnamed: 0,St. Mary's River,St. Clair River,Detroit River,Niagara River,Ottawa River,St. Lawrence River,Lake Superior,Lake Michigan and Lake Huron,Lake St. Clair,Lake Erie,Lake Ontario
0,2018-01-31,2066.563266,5063.051685,5371.705283,6320,2435,6229.705652,183.65,176.73,175.24,174.38,74.83
1,2018-02-28,2097.711794,5510.457818,6054.141220,6540,2407,8211.884723,183.58,176.74,175.32,174.44,74.90
2,2018-03-31,2507.456525,6294.834393,6866.834639,6900,2144,8976.439508,183.54,176.76,175.51,174.68,74.93
3,2018-04-30,2339.820809,6102.279855,6730.913789,7240,2535,8523.370006,183.47,176.79,175.60,174.77,75.01
4,2018-05-31,2368.137653,6124.933330,6385.448293,7400,4467,7730.498377,183.50,176.92,175.68,174.88,75.26


In [4]:
# Step 1: Rename 'Unnamed: 0' to 'Date'
data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)

# Step 2: Replace "---" with NaN
data.replace('---', float('nan'), inplace=True)

# Convert columns to appropriate data types
for col in data.columns[1:]:
    data[col] = data[col].astype(float)

# Step 3: Convert 'Date' column to datetime type
data['Date'] = pd.to_datetime(data['Date'])

# Preview the cleaned data
data.head()

,Date,St. Mary's River,St. Clair River,Detroit River,Niagara River,Ottawa River,St. Lawrence River,Lake Superior,Lake Michigan and Lake Huron,Lake St. Clair,Lake Erie,Lake Ontario
0,2018-01-31,2066.563266,5063.051685,5371.705283,6320.0,2435.0,6229.705652,183.65,176.73,175.24,174.38,74.83
1,2018-02-28,2097.711794,5510.457818,6054.141220,6540.0,2407.0,8211.884723,183.58,176.74,175.32,174.44,74.90
2,2018-03-31,2507.456525,6294.834393,6866.834639,6900.0,2144.0,8976.439508,183.54,176.76,175.51,174.68,74.93
3,2018-04-30,2339.820809,6102.279855,6730.913789,7240.0,2535.0,8523.370006,183.47,176.79,175.60,174.77,75.01
4,2018-05-31,2368.137653,6124.933330,6385.448293,7400.0,4467.0,7730.498377,183.50,176.92,175.68,174.88,75.26


In [5]:
# Check if there are any NaN values in the dataset
data.isnull().sum()

Date                            0
St. Mary's River                0
St. Clair River                 0
Detroit River                   0
Niagara River                   0
Ottawa River                    0
St. Lawrence River              0
Lake Superior                   0
Lake Michigan and Lake Huron    0
Lake St. Clair                  0
Lake Erie                       0
Lake Ontario                    0
dtype: int64

In [6]:
data.shape

(36, 12)

# Model

## water level threshold

In [16]:
# Python Code

def calculate_thresholds(df):
    threshold_values = []
    
    for column in df.columns:
        if column != 'Date':
            HWL = df[column].quantile(0.25)
            LWL = df[column].quantile(0.75)
            MWL = (HWL + LWL) / 2
            threshold_values.append([column, HWL, LWL, MWL])
            
    threshold_df = pd.DataFrame(threshold_values, columns=['Lake', 'HWL', 'LWL', 'MWL'])
    return threshold_df

thresholds = calculate_thresholds(data)
thresholds

,Lake,HWL,LWL,MWL
0,St. Mary's River,2366.792603,2624.121922,2495.457262
1,St. Clair River,6154.666016,6915.681195,6535.173605
2,Detroit River,6573.047384,7224.334793,6898.691089
3,Niagara River,6967.500000,7652.500000,7310.000000
4,Ottawa River,1722.250000,2511.000000,2116.625000
5,St. Lawrence River,8233.122356,9096.786094,8664.954225
6,Lake Superior,183.617500,183.770000,183.693750
7,Lake Michigan and Lake Huron,176.882500,177.282500,177.082500
8,Lake St. Clair,175.590000,175.882500,175.736250
9,Lake Erie,174.650000,174.935000,174.792500


## class weight

In [9]:
def weight(r, m, a, b):
    return a*r+b*m

In [10]:
a = 0.734
b = 0.266

In [11]:
r = pd.read_excel('r.xlsx')
m = pd.read_excel('m.xlsx')

In [12]:
w1 = []
w2 = []
w3 = []
for i in range(3):
    for j in range(12):
        w1.append(weight(r.iloc[i, 1], m.iloc[i, 1], a, b))
        w2.append(weight(r.iloc[i, 2], m.iloc[i, 2], a, b))
        w3.append(weight(r.iloc[i, 3], m.iloc[i, 3], a, b))

## Optimal Water Level

> write a function that realize max wi1(xi −HWLi)^2 +wi2(xi − MWLi)^2 +wi3(xi −LWLi)^2
s.t. mi ≤ xi ≤ Mi

In [13]:
from scipy.optimize import minimize

def optimize_xi(HWL, MWL, LWL, w1, w2, w3, mi, Mi):
    """Function to find xi that maximizes the equation given constraints."""
    
    # Define the function that needs to be maximized
    func = lambda xi: -(w1*(xi-HWL)**2 + w2*(xi-MWL)**2 + w3*(xi-LWL)**2)
    
    # Define the bounds for xi
    bounds = [(mi, Mi)]
    
    # Initial guess
    x0 = [mi]
    
    # Use scipy's minimize function to find the maximum
    result = minimize(func, x0, bounds=bounds)
    
    # Return the optimized xi
    return result.x[0]


> use optimize_xi to get the optimal water level of five lakes

In [17]:
# Let's select the five lakes
lakes = ["Lake Superior", "Lake Michigan and Lake Huron", "Lake St. Clair", "Lake Erie", "Lake Ontario"]

# And create an empty list to store the optimal water level of each lake
optimal_levels = []

# Now we iterate through the lakes, calculate their optimal water level and add it to our list
for lake in lakes:
    lake_data = thresholds[thresholds['Lake'] == lake]
    optimal_level = optimize_xi(
        lake_data['HWL'].values[0], 
        lake_data['MWL'].values[0], 
        lake_data['LWL'].values[0], 
        w1[0], 
        w2[0], 
        w3[0], 
        data[lake].min(), 
        data[lake].max())
    optimal_levels.append([lake, optimal_level])

# We convert our list to a DataFrame for better readability
optimal_levels_df = pd.DataFrame(optimal_levels, columns=['Lake', 'Optimal Water Level'])
optimal_levels_df

,Lake,Optimal Water Level
0,Lake Superior,183.47
1,Lake Michigan and Lake Huron,176.73
2,Lake St. Clair,175.24
3,Lake Erie,174.38
4,Lake Ontario,74.60


We have successfully computed the optimal water level for the five lakes. Here are the calculated optimal water levels:

  - Lake Superior: 183.47 meters
  - Lake Michigan and Lake Huron: 176.73 meters
  - Lake St. Clair: 175.24 meters
  - Lake Erie: 174.38 meters
  - Lake Ontario: 74.60 meters

These computations utilized the given water level thresholds (HWL, MWL, and LWL) and weights for each lake. Please note that since this is an optimization problem, the results can slightly vary due to the tolerance of the optimization algorithm.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=81c0e249-cf22-4e13-8945-32eaaed96e94' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>